In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
torch.manual_seed(1)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w4-p2/sample.csv
/kaggle/input/2021-ai-w4-p2/train.csv
/kaggle/input/2021-ai-w4-p2/test.csv


In [2]:
X = pd.read_csv('/kaggle/input/2021-ai-w4-p2/train.csv')
y = X['label']
X = X.drop(['label'], axis=1)
test = pd.read_csv('/kaggle/input/2021-ai-w4-p2/test.csv')

In [3]:
X.describe()

,N,P,K,temperature,humidity,ph,rainfall
count,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000,1650.000000
mean,50.873333,53.345455,48.026667,25.515033,71.665353,6.474123,102.509781
std,37.314088,33.200469,50.954520,5.035143,21.971147,0.777618,55.298782
min,0.000000,5.000000,5.000000,8.825675,14.273280,3.504752,20.360011
25%,20.250000,28.000000,20.000000,22.713912,60.521146,5.956121,63.659862
50%,37.000000,51.000000,31.000000,25.546615,80.519728,6.431830,93.516755
75%,85.000000,68.000000,48.000000,28.435373,89.894716,6.921558,122.340263
max,140.000000,145.000000,205.000000,43.675493,99.981876,9.935091,298.560117


In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21])

In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sc = StandardScaler()
X_sc = sc.fit_transform(X)
test_sc = sc.transform(test)

X_sc = torch.FloatTensor(X_sc)
test_sc = torch.FloatTensor(test_sc)

In [6]:
X = torch.FloatTensor(X.to_numpy())
test = torch.FloatTensor(test.to_numpy())
y = torch.LongTensor(y)

In [7]:
import torch.nn.functional as F

nb_class = 22
nb_data = len(y)

# 정보 4, class 3
W = torch.zeros((7, nb_class), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 변경
optimizer = optim.SGD([W, b], lr = 1)

nb_epochs = 100000
for epoch in range(nb_epochs + 1):
      
#     y_one_hot = torch.zeros(nb_data, nb_class)
#     # unsqueeze : 하나의 리스트에 들어있던 원소 -> 각각의 리스트로
#     # scatter_(dim, index, src) . dim=1 행방향, src는 scatter할 값
#     y_one_hot.scatter_(1, y.unsqueeze(1), 1)  
#     cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    loss = torch.nn.CrossEntropyLoss()
    hypothesis = X_sc.matmul(W) + b
    cost = loss(hypothesis, y)
    optimizer.zero_grad()
    cost.backward()   # cost 이용해 optimizer 알고리즘 수행
    optimizer.step()  # W 갱신

    if epoch % 5000 == 0:
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
  



Epoch    0/100000 Cost : 3.091014
Epoch 5000/100000 Cost : 0.107798
Epoch 10000/100000 Cost : 0.086161
Epoch 15000/100000 Cost : 0.076600
Epoch 20000/100000 Cost : 0.070934
Epoch 25000/100000 Cost : 0.067096
Epoch 30000/100000 Cost : 0.064283
Epoch 35000/100000 Cost : 0.062112
Epoch 40000/100000 Cost : 0.060374
Epoch 45000/100000 Cost : 0.058943
Epoch 50000/100000 Cost : 0.057739
Epoch 55000/100000 Cost : 0.056710
Epoch 60000/100000 Cost : 0.055816
Epoch 65000/100000 Cost : 0.055032
Epoch 70000/100000 Cost : 0.054336
Epoch 75000/100000 Cost : 0.053713
Epoch 80000/100000 Cost : 0.053152
Epoch 85000/100000 Cost : 0.052644
Epoch 90000/100000 Cost : 0.052179
Epoch 95000/100000 Cost : 0.051754
Epoch 100000/100000 Cost : 0.051361


In [8]:
hypothesis = F.softmax(X_sc.matmul(W)+b, dim=1)
predict = torch.argmax(hypothesis, dim=1) # argmax : 가장 큰 값의 인덱스 리턴
print(predict.shape)
print(y.shape)

correct_prediction = predict.float() == y
print(correct_prediction)
accuracy = correct_prediction.sum().item()/len(correct_prediction)
print('{:2.2f}%'.format(accuracy*100))

torch.Size([1650])
torch.Size([1650])
tensor([True, True, True,  ..., True, True, True])
98.06%


In [9]:
hypothesis = F.softmax(test_sc.matmul(W)+b, dim=1)
predict = torch.argmax(hypothesis, dim=1) 
predict = le.inverse_transform(predict)

In [10]:
submit = pd.read_csv('/kaggle/input/2021-ai-w4-p2/sample.csv')
submit.head()

,id,label
0,0,coffee
1,1,coffee
2,2,coffee
3,3,coffee
4,4,coffee


In [11]:
submit['label'] = predict


In [12]:
submit.to_csv('submit.csv', index=False)